In [4]:
import json
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import json

In [5]:
def splitFromFile(category_name, sample_type):
    ''' Get the Features and label from the JSON file

    :param category_name: The path of Train or Test
    :param sample_type: The concrete content of file
    :return: data[] and target[]
    '''

    target = []
    data = []
    category_name=os.path.join(category_name,sample_type)
    file_list = os.listdir(category_name)
    file_list = sorted(file_list, key=lambda x: int(x[0:x.index('.')]))
    for _ in file_list:
        with open(os.path.join(category_name,_), encoding='UTF-8') as fp:
            dic = json.load(fp)  # The size of content is only a line
            a = [
                  dic["add_annotation_line"]
                , dic["add_call_line"]
                , dic["add_classname_line"]
                , dic["add_condition_line"]
                , dic["add_field_line"]
                , dic["add_import_line"]
                , dic["add_packageid_line"]
                , dic["add_parameter_line"]
                , dic["add_return_line"]
                , dic["del_annotation_line"]
                , dic["del_call_line"]
                , dic["del_classname_line"]
                , dic["del_condition_line"]
                , dic["del_field_line"]
                , dic["del_import_line"]
                , dic["del_packageid_line"]
                , dic["del_parameter_line"]
                , dic["del_return_line"]
            ]
            label = 0 if dic['label'] == "NEGATIVE" else 1
            target.append(label)
            data.append(a)
            if sample_type=="TestSample" or sample_type=="TestSample_Rule":
                if dic["true_label"] is not None:
                    target_true.append(0 if dic['true_label'] == "NEGATIVE" else 1)
                else:
                    raise Exception("The true_label is null", _)
    return data, target

In [6]:
def train(x_train,y_train):
    #print(len(x_train), len(x_test))
    model=RandomForestClassifier()
    model.fit(x_train,y_train)
    return model

if __name__=="__main__":

    target_true=[]

    #启发规则过滤Training
    project_path=r"E:\Project_Code\JustInTestPro\Heuristic_Sample\commons-math"
    x_train_total,y_train_total=splitFromFile(project_path,"Train")
    try:
        x_test,y_test=splitFromFile(project_path,"TestSample_Rule")
    except Exception as ep:
        print(ep.__repr__())



    preprocessing.scale(x_train_total)
    preprocessing.scale(x_test)
    total_sum=0
    times=10
    accuracyList1=[]
    precisionList1=[]
    recallList1=[]
    accuracyList2=[]
    precisionList2=[]
    recallList2=[]
    for i in range(times):
        print("The times of iterator is %d" %(i+1))
        from imblearn.under_sampling import RandomUnderSampler
        rus = RandomUnderSampler()
        x_train, y_train = rus.fit_resample(x_train_total, y_train_total)
        model=train(x_train,y_train)
        y_pred=model.predict(x_test)
        accuracyList1.append(accuracy_score(y_test, y_pred))
        precisionList1.append(precision_score(y_test,y_pred))
        recallList1.append(recall_score(y_test,y_pred))
        accuracyList2.append(accuracy_score(target_true, y_pred))
        precisionList2.append(precision_score(target_true,y_pred))
        recallList2.append(recall_score(target_true,y_pred))
    for i in zip(accuracyList2,precisionList2,recallList2):
        print("The True Accuracy is {:.2%}; The True Precision is {:.2%}; The True Recall is {:.2%}".format(
            *i
        ))
    print("The average True Accuracy is {:.2%}; The True average Precision is {:.2%}; The True average Recall is {:.2%}".format(
        sum(accuracyList2)/times, sum(precisionList2)/times, sum(recallList2)/times)
    )

The times of iterator is 1
The times of iterator is 2
The times of iterator is 3
The times of iterator is 4
The times of iterator is 5
The times of iterator is 6
The times of iterator is 7
The times of iterator is 8
The times of iterator is 9
The times of iterator is 10
The True Accuracy is 75.28%; The True Precision is 65.99%; The True Recall is 68.17%
The True Accuracy is 75.28%; The True Precision is 65.45%; The True Recall is 69.97%
The True Accuracy is 72.95%; The True Precision is 62.33%; The True Recall is 67.57%
The True Accuracy is 74.94%; The True Precision is 64.58%; The True Recall is 71.17%
The True Accuracy is 73.28%; The True Precision is 63.14%; The True Recall is 66.37%
The True Accuracy is 74.17%; The True Precision is 63.66%; The True Recall is 69.97%
The True Accuracy is 74.61%; The True Precision is 64.29%; The True Recall is 70.27%
The True Accuracy is 74.83%; The True Precision is 65.14%; The True Recall is 68.47%
The True Accuracy is 73.39%; The True Precision i